# **실습 3. n-gram 모델을 바탕으로 새로운 텍스트 생성해보기**
본 실습에서 사용하는 데이터셋(shakespeare_input.txt)의 출처는 다음과 같음을 밝힌다.

https://github.com/karpathy/char-rnn/blob/master/data/tinyshakespeare/input.txt

## **0. 과제 이해하기**

실습 2에서는 파이썬을 활용하여 영어 텍스트를 글자와 단어 단위에서 분석하는 방법 및 토큰화의 방법을 배웠다. 이번 실습에서는 기본 언어 모델 중 하나인 n-gram 모델을 사용하여 주어진 본문을 분석하는 작업으로 확장해볼 것이다. 나아가, 본문을 분석함으로써 특정 텍스트 스타일을 파악해내고 모델을 훈련하는 실습을 진행하고자 한다. 우리의 최종 과제는 셰익스피어 스타일을 분석하고 이를 모델에 훈련시켜, 셰익스피어 스타일의 텍스트를 새롭게 창작해보는 활동으로, 꽤나 흥미로울 것이니 즐겁게 따라올 수 있기를 기대한다.



## **1. n-gram 언어 모델 만들기**

### **1.1 [글자-빈도] 형태로 변환하는 함수 만들기**

본격적으로 n-gram 언어 모델을 생성하기에 앞서, [letter -> count 쌍]에 대한 counter를 [letter -> frequency 쌍]으로 변환해주는 함수를 작성하고자 한다. 실습 2에서 작성했던 코드를 참조하여 <normalize> 함수를 만들되, 이번에는 character 내의 특수 문자를 제거하지 않도록 하라. 이번 실습 역시 collections 모듈의 Counter 클래스를 활용할 것이다.

In [ ]:
def normalize(counter):
    """ `letter -> count` counter를 (letter, frequency) 쌍의 list로 변환하라.
       이때 frequency의 내림차순 기준으로 정렬해야 한다.

    매개변수
    -----------
    counter : collections.Counter
        letter -> count

    반환 값
    -------
    List[Tuple[str, float]]
       A list of tuples - (letter, frequency) pairs in order
       of descending-frequency

    예시
    --------
    >>> from collections import Counter
    >>> letter_count = Counter({"a": 1, "b": 3})
    >>> letter_count
    Counter({'a': 1, 'b': 3})

    >>> normalize(letter_count)
    [('b', 0.75), ('a', 0.25)]
    """
    # 여기에 코드 작성

### **1.2 n-gram 분석 모델 생성하기**

이제부터는 파이썬의 collections 모듈을 본격적으로 사용할 것인데, 이때 Counter 클래스뿐 아니라 defaultdict 클래스도 사용할 것이다. 이를 기반으로 직접 함수를 만들어서, 텍스트에서 발생하는 n-그램을 실제로 분석해보도록 하자. 이때 토큰의 단위를 ‘n개 글자로 구성된 글자 시퀀스’로 설정하여 분석을 진행하면 된다. 위에서 언급했듯, 우리의 실습에서는 n개의 단어가 아닌 n개의 글자를 기준으로 토큰화할 것이다.

`model: {(n-1) character history -> {nth-letter -> count}}`



```
예시1: 텍스트 "cacao"를 3그램 단위로 분석하는 과정
history is "~~", next char is "c", increment model["~~"]["c"]
history is "~c", next char is "a", increment model["~c"]["a"]
history is "ca", next char is "c", increment model["ca"]["c"]
history is "ac", next char is "a", increment model["ac"]["a"]
history is "ca", next char is "o", increment model["ca"]["o"]
history is "ao", next char does not exists. End process

예시2: 텍스트 "cacao"를 4그램 단위로 분석하는 과정
history is "~~~", next char is "c", increment model["~~~"]["c"]
history is "~~c", next char is "a", increment model["~~c"]["a"]
history is "~ca", next char is "c", increment model["~ca"]["c"]
history is "cac", next char is "a", increment model["cac"]["a"]
history is "aca", next char is "o", increment model["aca"]["o"]
history is "cao", next char does not exists. End process
```



우리가 학습시킬 ‘n-gram 분석 모델’의 작동 원리를 살펴보자면, 

>① n-1개의 characters로 이루어진 시퀀스가 주어지고, 해당 시퀀스 뒤에 따라오는 character 히스토리에 대한 기록을 가지고 있다. (ex. "ca" 시퀀스 뒤에 "c"가 오더라)  
② ①의 [시퀀스-character] 히스토리가 몇 번씩 발생했는지 count하고, <normalize> 함수를 통해 각 히스토리 count값을 ‘빈도 값’으로 변환하여 확률 형태로 표현한다.  
③ 문서의 시작 부분에 더미 글자(dummy character) " ~ "를 활용하여 여백(padding)을 줄 것인데, 텍스트 도입 부분에 개의 “~”를 미리 채워두도록 한다.  

요약하자면, 우리가 학습시킬 모델은 
- 주어진 텍스트에서 (n-1) 길이의 모든 히스토리와
- 각 히스토리를 따르는 여러 character에 대한 기록들 모두 다룬다고 정리할 수 있으며,
 이에 따라 우리 모델을 일종의 ‘두 개의 중첩된 딕셔너리’로 생각해볼 수 있다. 

위의 조건을 충족시키기 위해, 우리의 모델은 defaultdict 클래스를 활용하여 default 딕셔너리(=기본 값으로서 ‘비어있는 Counter 인스턴스’를 취하는 딕셔너리) 형태로 만들어야 한다. 이를 통해 모델이 새로운 히스토리를 만날 때마다 그 히스토리에 대한 비어있는 counter 인스턴스를 생성하게 될 것이다. 이후 모델은 학습을 거치면서 비어있던 counter들을 ‘주어진 텍스트의 모든 히스토리’로 채워넣게 될 것이다. 
```
model = defaultdict(Counter)
model[history] # 해당 히스토리에 대한 counter를 반환
model[history][char] += 1 # 히스토리 뒤에 오는 글자를 바탕으로 counter를 업데이트
```
위 사항들을 모두 고려하여 <train_lm> 함수를 완성시켜 보아라.

In [ ]:
from collections import defaultdict

def train_lm(text, n):
    """ 글자 기반의 n-gram 언어 모델 훈련시키기.
        
    이것은 n-1개의 글자 시퀀스가 주어졌을 때, 시퀀스 내 n번째 글자에 대한 확률분포가 무엇인지 학습하게 될 것이다
    예를 들어, text = "cacao" 라는 텍스트를 훈련한다면,
    n 크기가 3이라고 할 때 그에 대한 딕셔너리가 반환될 것이다.
    우리가 주어진 히스토리에 대한 각 카운트를 정규화하는지 확인하라.
    
        {'ac': [('a', 1.0)],
         'ca': [('c', 0.5), ('o', 0.5)],
         '~c': [('a', 1.0)],
         '~~': [('c', 1.0)]}
여기서 "~"는, 어떠한 히스토리에 대해 (n-1)개의 글자가 앞에 존재하지 않더라도, 특정 글자가 나타날 확률값을 측정하기 위하여 히스토리를 임의로 채워넣기(패딩) 위해 사용된다. 
   우리가 훈련한 이 텍스트에 따르면 'ac'라는 수열을 볼 때
이 모델은 다음 글자가 100% 경우에 'a'여야 한다고 예측한다.
    
패딩을 만들기 위해서, 파이썬을 호출하여 반복된 시퀀스를 쉽게 만들어내도록 하자:
          `"p" * 4` returns `"pppp"`

    매개변수
    -----------
    text: str 
        하나의 문자열 (이 문자열은 소문자화하지 않아도 된다).
        
    n: int
        분석할 n-gram의 길이

    반환 값
    -------
    Dict[str, List[Tuple[str, float]]]
      {n-1 history -> [(letter, normalized count), ...]}

히스토리(n-1 길이의 문자열 형태)를 (char, prob) 쌍의 list에 매핑하는 딕셔너리. 
여기서 prob은 글자(char)가 특정 히스토리 뒤에 나타날 의 확률, 즉 빈도를 가리킨다

    예시
    --------
    >>> train_lm("cacao", 3)
    {'ac': [('a', 1.0)],
     'ca': [('c', 0.5), ('o', 0.5)],
     '~c': [('a', 1.0)],
     '~~': [('c', 1.0)]}
    """
    # 여기에 코드 작성

- “cacao"에 대하여 train_lm()을 테스트해보아라. (이때 n은 3으로 지정한다.) 

In [ ]:
# 여기에 코드 작성

- 이번에는 다른 예시를 가지고 테스트해보자.
이하 텍스트는 Dr.Seuss의 "The Cat in the Hat"로부터 발췌한 것이다.

In [ ]:
text = """The sun did not shine, it was too wet to play, 
so we sat in the house all that cold, cold wet day. 
I sat there with Sally. We sat here we two 
and we said 'How we wish we had something to do.'"""

# 여기에 코드 작성

- 다음은 글자 "T"에 대해 1.0의 확률을 나타내야 하는데, 이는 ‘T'가 이 모델이 접한 유일한 시작 글자이기 때문이다.  (i.e., with no history, indicated by "~~")

In [ ]:
lm3["~~"]

- 마찬가지로, 다음은 "h"에 대해 1.0의 확률을 보여야 하는데, 'h'가 이 모델이 "~T"의 히스토리 이후에 본 유일한 글자이기 때문이다.

In [ ]:
lm3["~T"]

- 마지막 예시는 글자 "e", "a", "", "i"의 확률 분포를 나타내야 하는데, 텍스트에서 이 네 개의 글자 모두 "th" 뒤에 따라오는 것으로 관찰되었기 때문이다.

In [ ]:
lm3["th"]

## **2. 텍스트 생성하기**
언어 모델에서 재미있는 것은, 학습된 확률 분포를 사용하여 글자를 새롭게 생성할 수 있다는 것이다. 다시 말해, 학습시킨 모델의 스타일대로 랜덤 텍스트를 만들어낼 수 있다.

먼저, <generate_letter> 함수를 만들어보자. 이 함수는 ① 특정 히스토리가 주어졌을 때 ② 언어 모델에 저장된 확률을 바탕으로 ③ 하나의 랜덤 글자를 가져오는 함수이다. 아래 두 개의 힌트 두 개를 활용하여 직접 함수의 코드를 완성해보자.

>힌트1: unzip 함수를 사용하라.
예: [(char0, prob0), (char1, prob1), ...] 
    -> ((char0, char1, ...), (prob0, prob1, ...))  

>힌트2: 여러 원소로 구성된 choices가 있다고 할 때, 우리는  <np.random.choice(choices, p=probabilities)>를 통해 각 원소의 발생 확률을 probabilities로 지정시킬 수 있다. 이후, 해당 확률에 따라 choices로부터 하나의 원소를 반환할 수 있다.   
예: <np.random.choice(["a", "b"], [0.25, 0.75])>는 25%의 확률로 "a"를. 75%의 확률로 "b"를 반환해줄 것이다.  

In [ ]:
def generate_letter(lm, history):
    """ 언어 모델에 저장된 지정된 히스토리와 관련된 확률 분포에 따라, 글자를 랜덤하게 선택합니다.
    주의: 모델 내에서 히스토리가 존재하지 않으면 더미 글자 "~"를 반환

    매개변수
    ----------
    lm: Dict[str, List[Tuple[str, float]]] 
        n-gram 언어모델
        (= 딕셔너리: history -> [(char, freq), ...])

    history: str
       다음 글자를 생성하기 위한 문맥/히스토리로 사용할 길이(n-1) 문자열

    반환 값
    -------
    str
        예측된 글자.
        '~': 언어 모델에 히스토리가 없는 경우
    """
    # 여기에 코드 작성

- 다음은 "e", "a", " ” 또는 "i"를 생성해야 하는데, 이들 글자는 "th" 뒤에 오는 유일한 네 글자이기 때문이다.

In [ ]:
# `generate_letter`를 100번 실행한 후 반환된 결과를 중복 없이 저장해두어라
set(generate_letter(lm3, "th") for i in range(100))

- 다음 코드는 예측되는 여러 개의 다음 글자를 생성해주는데, 이는 분포에 대한 감각을 얻기 위한 것이다. "e"는 "th"를 따를 확률이 높기 때문에, 평균적으로 다른 글자보다 더 자주 나타나야 한다.

In [ ]:
[generate_letter(lm3, "th") for i in range(10)]

- 마지막으로, 언어 모델에 따라 전체 텍스트 시퀀스를 생성한다. 시작 부분은 히스토리가 없는 상태, 즉 n-1 개의 '~'로 시작할 것이며, 랜덤 글자를 생성하고, 히스토리를 업데이트하고, 반복할 것이다. 이때 'nletters' 변수를 새로 만들어 랜덤 글자의 생성 개수를 입력해보자.

우리의 예시는,

```
history "~~" will generate 'T' (since only one possibility)  
history "~T" will generate 'h' (since only one possibility)  
history "Th" will generate 'e' (since only one possibility)  
history "he" could generate 'r' (out of multiple possibilities)  
history "er" will generate 'e' (since only one possibility)  
```



등이 있다. 지금까지 생성된 텍스트는 ”There"이 될 것이다.

In [ ]:
def generate_text(lm, n, nletters=100):
    """ n-gram 언어 모델 `lm`에 저장된 확률 분포로부터 추출하여, 텍스트의 'nletters'를 무작위로 생성한다.

    매개변수
    ----------
    lm: Dict[str, List[Tuple[str, float]]]
         n-gram 언어모델
        (= 딕셔너리: history -> [(char, freq), ...])
    
    n: int
        n-gram 모델을 구성하는 토큰 한 개의 길이(글자 n개)
    
    nletters: int
        무작위로 생성할 글자의 개수

    반환 값
    -------
    str
        모델 생성 텍스트.
       생성된 글자 수 'nletters'를 포함해야 하며, 미리 표시한 ‘~’들은 포함하지 않음.
    """
    # 여기에 코드 작성

- 다음 코드를 통해 "The Cat in the Hat"의 시작 부분에서 훈련한 3-gram 언어 모델에 따라, 40개의 글자를 생성할 수 있다. 물론, 히스토리 길이도 짧고 훈련 데이터의 양도 적기 때문에, 생성한 결과가 깔끔하지는 않을 것이다.

In [ ]:
print(generate_text(lm3, 3, 40))

## **3. 셰익스피어 스타일의 텍스트 생성해보기**

지금까지, 텍스트 분석을 통해 특정 텍스트 스타일을 학습하고 해당 스타일이 적용된 언어를 생성하는 방법을 배웠다. 여러분은 마지막으로 셰익스피어 스타일의 새로운 텍스트를 직접 만들어보는 활동을 할 것이다. 다음 셀을 통해 Andrej Karpathy의 shakespeare_input.txt 파일을 로드할 수 있다.


In [ ]:
path_to_shakespeare = "./dat/shakespeare_input.txt"
with open(path_to_shakespeare, "r") as f:
    shakespeare = f.read()
print(str(len(shakespeare)) + " character(s)")
chars = set(shakespeare)
print(f"'~' is a good pad character: {'~' not in chars}")

이제 다양한 n값(ex. 3, 5, 11)에 대한 훈련 모델을 실험하고, 약 500자 정도의 텍스트를 생성해볼 것이다. 이때 사용하는 n-gram 모델은 11-gram이 적합할 것이다.

In [ ]:
# 여기에 코드 작성

In [ ]:
print(generate_text(lm3, 3, 500))

In [ ]:
# 여기에 코드 작성

In [ ]:
print(generate_text(lm5, 5, 500))

In [ ]:
# 여기에 코드 작성

In [ ]:
print(generate_text(lm11, 11, 500))

보너스 아이디어: 인터넷에서 모델을 훈련시킬 다른 텍스트(ex. 노래 가사, 특정 작가의 책 등)도 함께 찾아보고 적용해보면, 색다른 경험이 될 것이다.